In [1]:
import pandas as pd
import re
import os
from tqdm.notebook import tqdm

import numpy as np

In [2]:
os.getcwd()

'/home/gvisona/Projects/SelfPeptides/experimental_notebooks'

In [3]:
embeddings = np.load("../processed_data/HLA_embeddings/hla_proteins_ProtT5_embeddings.npy")
embeddings.shape

(25146, 1024)

# Load HLA references

In [22]:
# IPD IGMT
ipd_igmt_df = pd.read_csv("../processed_data/HLA_embeddings/hla_proteins_features.csv")
# Remove anormal alleles indicated with N, L Q, S
# ipd_igmt_df = ipd_igmt_df[~ipd_igmt_df["HLA_allele"].str.endswith("N")]
# ipd_igmt_df = ipd_igmt_df[~ipd_igmt_df["HLA_allele"].str.endswith("L")]
# ipd_igmt_df = ipd_igmt_df[~ipd_igmt_df["HLA_allele"].str.endswith("Q")]
# ipd_igmt_df = ipd_igmt_df[~ipd_igmt_df["HLA_allele"].str.endswith("S")]
# ipd_igmt_df.columns = 

ipd_igmt_df

,HLA_id,HLA_allele,Bp
0,HLA:HLA00001,A*01:01:01:01,365
1,HLA:HLA02169,A*01:01:01:02N,200
2,HLA:HLA14798,A*01:01:01:03,365
3,HLA:HLA15760,A*01:01:01:04,365
4,HLA:HLA16415,A*01:01:01:05,365
...,...,...,...
25141,HLA:HLA32558,C*18:15,366
25142,HLA:HLA33923,C*18:16,366
25143,HLA:HLA36442,C*18:17,366
25144,HLA:HLA36862,C*18:18,366


In [24]:
ipd_igmt_df[ipd_igmt_df["HLA_allele"].str.contains(":")]

,HLA_id,HLA_allele,Bp
0,HLA:HLA00001,A*01:01:01:01,365
1,HLA:HLA02169,A*01:01:01:02N,200
2,HLA:HLA14798,A*01:01:01:03,365
3,HLA:HLA15760,A*01:01:01:04,365
4,HLA:HLA16415,A*01:01:01:05,365
...,...,...,...
25141,HLA:HLA32558,C*18:15,366
25142,HLA:HLA33923,C*18:16,366
25143,HLA:HLA36442,C*18:17,366
25144,HLA:HLA36862,C*18:18,366


In [6]:
# Pseudosequences
ps_df = pd.read_csv("../data/NetMHCpan_pseudoseq/MHC_pseudo.dat", sep='\t')
ps_df

,HLA,sequence
0,HLA-A01:01,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
1,HLA-A01:02,YSAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
2,HLA-A01:03,YFAMYQENMAHTDANTLYIMYRDYTWVARVYRGY
3,HLA-A01:06,YFAMYQENMAHTDANTLYIIYRDYTWVALAYRGY
4,HLA-A01:07,YFAMYQENVAHTDENTLYIIYRDYTWVARVYRGY
...,...,...
3717,BoLA-6:01501,YYTKYREISENWYEANLYLLYTFYSMADQNYRGY
3718,BoLA-6:01502,YYTKYREISENWYEANLYLQFTFYSMADQNYRGY
3719,BoLA-6:03401,YHTKYREISENVYGSNLYLLYTFYSMADRNYRGY
3720,BoLA-6:04001,YSEMYEERAGIVFVNTLYLWCWFYSMAAGKYTWY


In [52]:
ps_df["sequence"].value_counts()

YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTWY    102
YSAMYEEKVAHTDENIAYLMFHYYTWAVQAYTGY     61
YFAMYQENVAQTDVDTLYIIYRDYTWAELAYTWY     45
YDSGYREKYRQADVSNLYLRSDSYTLAALAYTWY     45
YDSGYRENYRQADVSNLYLRYDSYTLAALAYTWY     44
                                     ... 
YYATYRNIFTNTYESNLYIRYDSYTWAVLTYTWY      1
YYATYRNIFTNTYESNLYIRYDFYTWAVLAYEWY      1
YYATYRNIFTNTYESNLYLSYNYYTWAVLAYEWY      1
YYATYRNIFTNTDESNLYIRYDSYTWAVLAYLWY      1
YYIVYQEKADHFFLGTLYLWYWFYSWAVQNYTWY      1
Name: sequence, Length: 2113, dtype: int64

# Load datasets

In [25]:
# immunogenicity IEDB
iedb_df = pd.read_csv("../processed_data/Immunogenicity/Processed_TCell_IEDB_Beta_noPrior.csv")
iedb_df

,Peptide,HLA,Qualitative Measurement,Number of Subjects Tested,Number of Subjects Positive,Peptide length,Alpha,Beta,Distr. Mean,Distr. Variance
0,CYTWNQMNL,HLA-A24:02,Positive,15.0,5.0,9,5.0,10.0,0.333333,0.013889
1,GYDQIMPKK,HLA-A24:02,Positive,1.0,1.0,9,1.0,0.0,1.000000,0.000000
2,GYDQIMPKK,HLA-A24:02,Positive,5.0,5.0,9,5.0,0.0,1.000000,0.000000
3,YLSGANLNL,HLA-A02:01,Positive,4.0,4.0,9,4.0,0.0,1.000000,0.000000
4,YLSGANLNL,HLA-A02:01,Positive,4.0,2.0,9,2.0,2.0,0.500000,0.050000
...,...,...,...,...,...,...,...,...,...,...
38713,WNQMNLGAT,HLA-A02:01,Negative,2.0,0.0,9,0.0,2.0,0.000000,0.000000
38714,NLGATLKGV,HLA-A02:01,Negative,2.0,0.0,9,0.0,2.0,0.000000,0.000000
38715,VLDFAPPGA,HLA-A02:01,Positive,5.0,4.0,9,4.0,1.0,0.800000,0.026667
38716,NLVPMVATV,HLA-A02:01,Positive,60.0,35.0,9,35.0,25.0,0.583333,0.003985


In [31]:
iedb_df[iedb_df["HLA"].str.contains("[A-Z]$", regex=True)]

,Peptide,HLA,Qualitative Measurement,Number of Subjects Tested,Number of Subjects Positive,Peptide length,Alpha,Beta,Distr. Mean,Distr. Variance


In [32]:
dhlap_imm_df = pd.read_csv("../processed_data/Immunogenicity/DHLAP_immunogenicity_data.csv")
dhlap_imm_df

,HLA,Peptide,Label
0,HLA-B40:01,SERDNYITLY,0
1,HLA-B35:01,MAILGDTAW,1
2,HLA-B27:05,KRWIILGLNK,1
3,HLA-A11:01,RVIKRKFNK,0
4,HLA-A24:02,RYPLTFGW,1
...,...,...,...
32780,HLA-A02:01,IIINGDIAL,0
32781,HLA-B37:01,IEWTGDGLY,0
32782,HLA-A02:01,ILDSFEPLR,0
32783,HLA-A03:01,VLCPKNMIIK,0


In [33]:
dhlap_imm_df[dhlap_imm_df["HLA"].str.contains("[A-Z]$", regex=True)]

,HLA,Peptide,Label


In [40]:
dhlap_ba_df = pd.read_csv("../processed_data/Binding_Affinity/DHLAP_binding_affinity_data.csv")
dhlap_ba_df

,HLA,Peptide,Method,Measurement,Type,Affinity(nM),Label
0,HLA-B08:01,HMMVIFRLM,purified MHC/competitive/fluorescence,half maximal inhibitory concentration (IC50),Negative,3439140.0,0
1,HLA-B08:01,NFLIKFLLI,purified MHC/competitive/fluorescence,half maximal inhibitory concentration (IC50),Negative,1606610.0,0
2,HLA-A24:02,SSKYYIKNI,purified MHC/competitive/fluorescence,half maximal inhibitory concentration (IC50),Negative,1449300.0,0
3,HLA-A11:01,KLYKNKSKQ,purified MHC/competitive/fluorescence,half maximal inhibitory concentration (IC50),Negative,1360000.0,0
4,HLA-A02:01,AMEKSSKYY,purified MHC/competitive/fluorescence,half maximal inhibitory concentration (IC50),Negative,1166430.0,0
...,...,...,...,...,...,...,...
327173,HLA-A11:01,SVQMTLSK,purified MHC,qualitative binding,Positive,0.0,1
327174,HLA-A02:09,TLGIVCPI,cellular MHC/direct/fluorescence,qualitative binding,Positive,0.0,1
327175,HLA-A11:01,TTSPIPLK,purified MHC,qualitative binding,Positive,0.0,1
327176,HLA-A03:01,TVTTITVY,cellular MHC/direct/fluorescence,qualitative binding,Positive,0.0,1


In [41]:
dhlap_ba_df = dhlap_ba_df[~dhlap_ba_df["HLA"].str.contains("[A-Z]$", regex=True)].reset_index(drop=True)
dhlap_ba_df

,HLA,Peptide,Method,Measurement,Type,Affinity(nM),Label
0,HLA-B08:01,HMMVIFRLM,purified MHC/competitive/fluorescence,half maximal inhibitory concentration (IC50),Negative,3439140.0,0
1,HLA-B08:01,NFLIKFLLI,purified MHC/competitive/fluorescence,half maximal inhibitory concentration (IC50),Negative,1606610.0,0
2,HLA-A24:02,SSKYYIKNI,purified MHC/competitive/fluorescence,half maximal inhibitory concentration (IC50),Negative,1449300.0,0
3,HLA-A11:01,KLYKNKSKQ,purified MHC/competitive/fluorescence,half maximal inhibitory concentration (IC50),Negative,1360000.0,0
4,HLA-A02:01,AMEKSSKYY,purified MHC/competitive/fluorescence,half maximal inhibitory concentration (IC50),Negative,1166430.0,0
...,...,...,...,...,...,...,...
327140,HLA-A11:01,SVQMTLSK,purified MHC,qualitative binding,Positive,0.0,1
327141,HLA-A02:09,TLGIVCPI,cellular MHC/direct/fluorescence,qualitative binding,Positive,0.0,1
327142,HLA-A11:01,TTSPIPLK,purified MHC,qualitative binding,Positive,0.0,1
327143,HLA-A03:01,TVTTITVY,cellular MHC/direct/fluorescence,qualitative binding,Positive,0.0,1


In [42]:
dhlap_ba_df.to_csv("../processed_data/Binding_Affinity/DHLAP_binding_affinity_data.csv", index=False)

In [43]:
# Hla Ligand Atlas
hlala_df = pd.read_csv("../processed_data/Binding_Affinity/HLA_Ligand_Atlas_processed.csv")
# hlala_df["HLA"] = "HLA-" + hlala_df["HLA"].str.replace("*", "", regex=False)
hlala_df

,Peptide,HLA,Label
0,KLFNEHIEAL,HLA-A11:01,0
1,KLFNEHIEAL,HLA-B40:02,0
2,KLFNEHIEAL,HLA-B58:01,0
3,KLFNEHIEAL,HLA-C02:02,0
4,KLFNEHIEAL,HLA-C07:01,0
...,...,...,...
724635,IAVDVTSAL,HLA-A11:01,0
724636,IAVDVTSAL,HLA-B15:01,0
724637,IAVDVTSAL,HLA-B35:01,1
724638,IAVDVTSAL,HLA-C03:03,1


# Homogenizing Format

We adopt the format HLA-A01:02 (no asterisk and HLA prefix)

In [44]:
ipd_igmt_alleles = set("HLA-" + ipd_igmt_df["HLA_allele"].str.replace("*","", regex=False))
# ipd_igmt_alleles

In [45]:
ps_alleles = set(ps_df["HLA"])
# ps_alleles

In [46]:
datasets_alleles = set(hlala_df["HLA"]).union(set(dhlap_ba_df["HLA"])).union(set(dhlap_imm_df["HLA"])).union(set(iedb_df["HLA"]))
len(datasets_alleles)

194

In [47]:
# Not in PS:
missing_ps_hla = [a for a in datasets_alleles if a not in ps_alleles]
len(missing_ps_hla)

5

In [48]:
missing_ps_hla

['HLA-A3', 'HLA-B44:01', 'HLA-B08:011', 'HLA-A1', 'HLA-A11']

In [49]:
# Not in IPD:
missing_prot_hla = set([a for a in datasets_alleles if a not in ipd_igmt_alleles])
len(missing_prot_hla)

159

In [50]:
missing_prot_hla


{'HLA-A01:01',
 'HLA-A01:03',
 'HLA-A02:01',
 'HLA-A02:02',
 'HLA-A02:03',
 'HLA-A02:04',
 'HLA-A02:05',
 'HLA-A02:06',
 'HLA-A02:07',
 'HLA-A02:08',
 'HLA-A02:09',
 'HLA-A02:11',
 'HLA-A02:17',
 'HLA-A02:20',
 'HLA-A03:01',
 'HLA-A03:02',
 'HLA-A1',
 'HLA-A11',
 'HLA-A11:01',
 'HLA-A11:02',
 'HLA-A23:01',
 'HLA-A24:02',
 'HLA-A24:03',
 'HLA-A24:07',
 'HLA-A24:13',
 'HLA-A25:01',
 'HLA-A26:01',
 'HLA-A26:02',
 'HLA-A26:03',
 'HLA-A26:08',
 'HLA-A29:01',
 'HLA-A29:02',
 'HLA-A3',
 'HLA-A30:01',
 'HLA-A30:02',
 'HLA-A30:04',
 'HLA-A31:01',
 'HLA-A32:01',
 'HLA-A33:01',
 'HLA-A33:03',
 'HLA-A34:01',
 'HLA-A34:02',
 'HLA-A66:01',
 'HLA-A66:02',
 'HLA-A68:01',
 'HLA-A68:02',
 'HLA-A69:01',
 'HLA-A74:01',
 'HLA-A80:01',
 'HLA-B07:02',
 'HLA-B07:06',
 'HLA-B08:01',
 'HLA-B08:011',
 'HLA-B13:01',
 'HLA-B13:02',
 'HLA-B14:01',
 'HLA-B14:02',
 'HLA-B15:01',
 'HLA-B15:02',
 'HLA-B15:03',
 'HLA-B15:08',
 'HLA-B15:09',
 'HLA-B15:10',
 'HLA-B15:11',
 'HLA-B15:13',
 'HLA-B15:16',
 'HLA-B15:17',
 'HLA

In [18]:
dhlap_ba_df[dhlap_ba_df["HLA"]=="HLA-B08:011"]

,HLA,Peptide,Method,Measurement,Type,Affinity(nM),Label


# Match and include partial prots

In [51]:
pseq_embeddings = np.load("../processed_data/HLA_embeddings/hla_pseudoseqs_ProtT5_embeddings.npy")
pseq_embeddings.shape

(3722, 1024)